In [28]:
!pip install pytorch-ignite
! pip install tensorboard

  Obtaining dependency information for tensorboard from https://files.pythonhosted.org/packages/6e/0c/1059a6682cf2cc1fcc0d5327837b5672fe4f5574255fa5430d0a8ceb75e9/tensorboard-2.15.1-py3-none-any.whl.metadata
  Obtaining dependency information for absl-py>=0.4 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio>=1.48.2 from https://files.pythonhosted.org/packages/bc/e5/f656b17fe1ccda1e2a4fe20298b8bcf7c804561c90ee763e39efc1c3772f/grpcio-1.59.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for google-auth<3,>=1.6.3 from https://files.pythonhosted.org/packages/86/a7/75911c13a242735d5aeaca6a272da380335ff4ba5f26d6b2ae20ff682d13/google_auth-2.23.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib<2,>=0.5 from https://files.pythonhosted.org/packages/ce/33/a907b4b67245647746dde8d61e1643ef

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification,AutoFeatureExtractor,AdamW
from tqdm import tqdm
import torch
import numpy as np
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import gym
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt


%matplotlib inline

In [4]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():

    print("CUDA is available. Using GPU.")
else:

    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU.


In [5]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# Load pre-trained feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Step 1: Modify classification head for the new number of classes
num_classes = 10  # Update with your new number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
model.config.image_size=32
# Move the model to the desired device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sushant\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [6]:
index_to_label = {
    0: 'Airplane',
    1: 'Automobile',
    2: 'Bird',
    3: 'Cat',
    4: 'Deer',
    5: 'Dog',
    6: 'Frog',
    7: 'Horse',
    8: 'Ship',
    9: 'Truck'
}

In [7]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy,Loss

In [8]:
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine

In [9]:
# Create the dataset and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
model = model  # Initialize your ViT model here
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:

def score_function(engine):
    return engine.state.metrics["accuracy"]
val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion)
}

In [22]:


from ignite.engine import Engine


def train_step(engine, batch):
    model.train()
    inputs, targets = batch
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    loss = criterion(probabilities, targets)
    loss.backward()
    optimizer.step()
    return loss.item()
# Create Ignite trainer and evaluator
trainer = Engine(train_step)


In [23]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y= batch
        y_pred = model(x.to(device))

    return y_pred, y.to(device)


evaluator = Engine(validation_step)

In [24]:
from ignite.engine import Events

validate_every = 10


@trainer.on(Events.EPOCH_COMPLETED(every=validate_every))
def run_validation():
    evaluator.run(test_loader)

In [25]:
@trainer.on(Events.EPOCH_COMPLETED(every=validate_every))
def log_validation():
    metrics = evaluator.state.metrics
    print(f"Epoch: {trainer.state.epoch},  Accuracy: {metrics['accuracy']}")
from ignite.metrics import Accuracy

Accuracy().attach(evaluator, "accuracy")

In [ ]:
!pip install jupyter-tensorboard


In [15]:
%load_ext tensorboard

In [26]:
model_checkpoint = ModelCheckpoint(
    "checkpoint",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer),
)

evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})

In [29]:
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine
tb_logger = TensorboardLogger(log_dir="tb-logger")

tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)






In [30]:
# Reinforcement Learning Setup
reward_window = []
best_accuracy = 0.0

@trainer.on(Events.EPOCH_COMPLETED)
def update_with_reward(engine):
    print("done")
    global best_accuracy

    # Use the validation accuracy as the reward
    reward = evaluator.state.metrics['accuracy']

    # Update the model based on the reward
    if reward > best_accuracy:
        best_accuracy = reward
        print(f"Updating model with reward: {reward}")
        torch.save(model.state_dict(), "best_model.pth")

# Run the training loop
trainer.run(train_loader, max_epochs=30)
tb_logger.close()

In [ ]:
%tensorboard --logdir=./tb-logger